In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sqlite3
import os

# Create sample time series data
date_rng = pd.date_range(start='2023-01-01', end='2023-01-10', freq='D')
data = {
    "date": date_rng,
    "value": np.random.randint(0, 100, size=(len(date_rng)))
}
df = pd.DataFrame(data)
df.loc[2, 'date'] = pd.NaT  # Introduce a NaT value
df.loc[4, 'date'] = 'error'  # Introduce an error value

# Convert the date column to string format
df['date'] = df['date'].astype(str)

# Set the SQLite database file path
db_path = os.path.join(os.path.expanduser('~'), 'test_db.sqlite')

# Save DataFrame to SQLite
con = sqlite3.connect(db_path)
df.to_sql('sample_data', con, if_exists='replace', index=False)

# SQL query
query = """
SELECT
  date AS OriginalDate,
  COALESCE(
    CASE
      WHEN date = 'NaT' THEN NULL
      WHEN (substr(date, 5, 1) = '-' AND substr(date, 8, 1) = '-') OR length(date) = 19 THEN date
      ELSE NULL
    END,
    datetime(DATE('now'), 'start of day')
  ) AS ProcessedDate
FROM sample_data;
"""

# Read query results and display side by side
result_df = pd.read_sql_query(query, con)
print(result_df)

con.close()


          OriginalDate        ProcessedDate
0  2023-01-01 00:00:00  2023-01-01 00:00:00
1  2023-01-02 00:00:00  2023-01-02 00:00:00
2                  NaT  2023-03-15 00:00:00
3  2023-01-04 00:00:00  2023-01-04 00:00:00
4                error  2023-03-15 00:00:00
5  2023-01-06 00:00:00  2023-01-06 00:00:00
6  2023-01-07 00:00:00  2023-01-07 00:00:00
7  2023-01-08 00:00:00  2023-01-08 00:00:00
8  2023-01-09 00:00:00  2023-01-09 00:00:00
9  2023-01-10 00:00:00  2023-01-10 00:00:00
